# Features in installment_payments

In [111]:
### set up system path to access helpers functions

import sys
import os

# Get the path to the 'src' directory
src_path = os.path.abspath(os.path.join(os.getcwd(), '../src'))

# Add the 'src' folder to sys.path
if src_path not in sys.path:
  sys.path.append(src_path)

In [112]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from helpers import *

In [113]:
# import the data

df = pd.read_csv('../data/installments_payments.csv')


In [114]:
# check the df

df.head()

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130,24350.130
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585


In [115]:
# save the SK_ID_PREV and SK_ID_CURR into a dataframe

engineered_install = df[['SK_ID_PREV','SK_ID_CURR']]

# check the dataframe for duplicates

engineered_install.duplicated().sum()

np.int64(12607649)

In [116]:
engineered_install.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13605401 entries, 0 to 13605400
Data columns (total 2 columns):
 #   Column      Dtype
---  ------      -----
 0   SK_ID_PREV  int64
 1   SK_ID_CURR  int64
dtypes: int64(2)
memory usage: 207.6 MB


In [117]:
# drop duplicates, keep 1

engineered_install.drop_duplicates(keep='first', inplace=True)

engineered_install.info()

<class 'pandas.core.frame.DataFrame'>
Index: 997752 entries, 0 to 13604682
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype
---  ------      --------------   -----
 0   SK_ID_PREV  997752 non-null  int64
 1   SK_ID_CURR  997752 non-null  int64
dtypes: int64(2)
memory usage: 22.8 MB


/var/folders/bh/crnltg950_n8l07mfm97qdtc0000gn/T/ipykernel_73415/117830804.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  engineered_install.drop_duplicates(keep='first', inplace=True)


In [118]:
# check all observations for 1 random previous loan

df[df['SK_ID_PREV'] == 1054186]

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.36,6948.36
47438,1054186,161674,1.0,2,-1300.0,-1307.0,6948.36,6948.36
138614,1054186,161674,1.0,8,-1120.0,-1175.0,6948.36,6948.36
216905,1054186,161674,1.0,12,-1000.0,-1033.0,6935.22,6935.22
425089,1054186,161674,1.0,3,-1270.0,-1275.0,6948.36,6948.36
577850,1054186,161674,1.0,1,-1330.0,-1338.0,6948.36,6948.36
1038640,1054186,161674,1.0,10,-1060.0,-1125.0,6948.36,6948.36
1128187,1054186,161674,1.0,7,-1150.0,-1175.0,6948.36,6948.36
1402152,1054186,161674,1.0,11,-1030.0,-1063.0,6948.36,6948.36
1608829,1054186,161674,1.0,9,-1090.0,-1154.0,6948.36,6948.36


### New feature 1. payment on installment due ratio

In [119]:
# payment on installment ratio = AMT_PAYMENT / AMT_INSTALMENT

df['payment_installment_ratio'] = df['AMT_PAYMENT'] / df['AMT_INSTALMENT']
df['payment_installment_ratio'] = df['payment_installment_ratio'].fillna(1)
df['payment_installment_ratio'] = df['payment_installment_ratio'].replace([np.inf, -np.inf],0)

df['payment_installment_ratio']


0           1.000000
1           1.000000
2           1.000000
3           1.000000
4           0.997942
              ...   
13605396    1.000000
13605397    1.000000
13605398    1.000000
13605399    1.000000
13605400    1.000000
Name: payment_installment_ratio, Length: 13605401, dtype: float64

In [120]:
# groupby SK_ID_PREV, get the average

avg_installment_payment_ratio = pd.DataFrame(df.groupby('SK_ID_PREV')['payment_installment_ratio'].mean()).reset_index()
avg_installment_payment_ratio.columns = ['SK_ID_PREV', 'installment_avg_payment_ratio']
avg_installment_payment_ratio

,SK_ID_PREV,installment_avg_payment_ratio
0,1000001,1.000000
1,1000002,1.000000
2,1000003,1.000000
3,1000004,1.000000
4,1000005,0.909091
...,...,...
997747,2843495,1.000000
997748,2843496,0.941176
997749,2843497,1.000000
997750,2843498,1.000000


In [121]:
# create a df to gather the engineered features

engineered_install = pd.merge(engineered_install, avg_installment_payment_ratio, on="SK_ID_PREV", how='outer')
engineered_install

,SK_ID_PREV,SK_ID_CURR,installment_avg_payment_ratio
0,1000001,158271,1.000000
1,1000002,101962,1.000000
2,1000003,252457,1.000000
3,1000004,260094,1.000000
4,1000005,176456,0.909091
...,...,...,...
997747,2843495,260963,1.000000
997748,2843496,425374,0.941176
997749,2843497,451578,1.000000
997750,2843498,393881,1.000000


### New feature 2. difference in when installment due and when actually paid

In [122]:
# install_diff_due_paid = DAYS_INSTALMENT - DAYS_ENTRY_PAYMENT

df['install_diff_due_paid'] = df['DAYS_INSTALMENT'] - df['DAYS_ENTRY_PAYMENT']
df['install_diff_due_paid'] = df['install_diff_due_paid'].fillna(0)
df['install_diff_due_paid'].value_counts()

install_diff_due_paid
 0.0       3149255
 1.0        525380
 2.0        507963
 3.0        459365
 9.0        454549
            ...   
-1899.0          1
 559.0           1
-1785.0          1
-996.0           1
-1817.0          1
Name: count, Length: 2404, dtype: int64

In [123]:
# use groupby to average per loan

avg_install_diff_due_paid = pd.DataFrame(df.groupby('SK_ID_PREV')['install_diff_due_paid'].mean()).reset_index()
avg_install_diff_due_paid.columns = ['SK_ID_PREV', 'install_avg_diff_due_paid']
avg_install_diff_due_paid


,SK_ID_PREV,install_avg_diff_due_paid
0,1000001,16.000000
1,1000002,19.750000
2,1000003,15.333333
3,1000004,26.714286
4,1000005,8.454545
...,...,...
997747,2843495,3.857143
997748,2843496,4.000000
997749,2843497,2.900000
997750,2843498,12.666667


In [124]:
# merge into the engineered features dataframe

engineered_install = pd.merge(engineered_install, avg_install_diff_due_paid, on='SK_ID_PREV', how='outer')
engineered_install

,SK_ID_PREV,SK_ID_CURR,installment_avg_payment_ratio,install_avg_diff_due_paid
0,1000001,158271,1.000000,16.000000
1,1000002,101962,1.000000,19.750000
2,1000003,252457,1.000000,15.333333
3,1000004,260094,1.000000,26.714286
4,1000005,176456,0.909091,8.454545
...,...,...,...,...
997747,2843495,260963,1.000000,3.857143
997748,2843496,425374,0.941176,4.000000
997749,2843497,451578,1.000000,2.900000
997750,2843498,393881,1.000000,12.666667


### New feature 3. 

In [125]:
# check the values in NUM_INSTALMENT_VERSION

df['NUM_INSTALMENT_VERSION'].value_counts()

NUM_INSTALMENT_VERSION
1.0      8485004
0.0      4082498
2.0       620283
3.0       237063
4.0        55274
          ...   
178.0          1
58.0           1
54.0           1
53.0           1
57.0           1
Name: count, Length: 65, dtype: int64

In [126]:
# check the row with the max value

df[df['NUM_INSTALMENT_VERSION'] == df['NUM_INSTALMENT_VERSION'].max()]

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT,payment_installment_ratio,install_diff_due_paid
7580806,1488056,276526,178.0,2,-150.0,NaN,347349.6,NaN,1.0,0.0


In [127]:
# check all rows for this loan

df[df['SK_ID_PREV'] == 1488056]

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT,payment_installment_ratio,install_diff_due_paid
7580806,1488056,276526,178.0,2,-150.0,NaN,347349.600,NaN,1.0,0.0
7601497,1488056,276526,1.0,1,-180.0,NaN,15269.715,NaN,1.0,0.0


- it looks like no payments were ever entered, and there were only 2 installments
- this is either bad data, or there was something unique about this loan

In [128]:
# sort by version to get the id for for another large value to check the data

df[['SK_ID_PREV','NUM_INSTALMENT_VERSION']].sort_values(by='NUM_INSTALMENT_VERSION')

,SK_ID_PREV,NUM_INSTALMENT_VERSION
10669485,1426047,0.0
10292353,1551198,0.0
4367655,2181116,0.0
4367654,2181116,0.0
10292356,1776445,0.0
...,...,...
11305632,2191808,72.0
9850838,2191808,72.0
9059074,2191808,72.0
11324478,2191808,73.0


In [129]:
# check the id with second largest value

df[df['SK_ID_PREV'] == 2191808]

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT,payment_installment_ratio,install_diff_due_paid
9044615,2191808,376539,72.0,4,-2831.0,-2951.0,7608.96,7608.96,1.0,120.0
9059074,2191808,376539,72.0,6,-2771.0,-2924.0,7608.96,7608.96,1.0,153.0
9437745,2191808,376539,72.0,1,-2921.0,-2921.0,7608.96,7608.96,1.0,0.0
9850838,2191808,376539,72.0,2,-2891.0,-2891.0,7608.96,7608.96,1.0,0.0
10427815,2191808,376539,72.0,7,-2741.0,-2902.0,7608.96,7608.96,1.0,161.0
11290270,2191808,376539,72.0,5,-2801.0,-2951.0,7608.96,7608.96,1.0,150.0
11305632,2191808,376539,72.0,3,-2861.0,-2951.0,7608.96,7608.96,1.0,90.0
11324478,2191808,376539,73.0,8,-2711.0,-2740.0,36710.10,36710.10,1.0,29.0


- the installment version number started at 72, not sure what happened before that

In [130]:
# calculate how many times a version number changes per loan

df.groupby('SK_ID_PREV')['NUM_INSTALMENT_VERSION'].nunique().describe()

count    997752.000000
mean          1.564032
std           1.200126
min           1.000000
25%           1.000000
50%           1.000000
75%           2.000000
max          59.000000
Name: NUM_INSTALMENT_VERSION, dtype: float64

In [131]:
# get ID of the max

df.groupby('SK_ID_PREV')['NUM_INSTALMENT_VERSION'].nunique()[df.groupby('SK_ID_PREV')['NUM_INSTALMENT_VERSION'].nunique() == 59]

SK_ID_PREV
1395269    59
Name: NUM_INSTALMENT_VERSION, dtype: int64

In [132]:
# check this loan

df[df['SK_ID_PREV'] == 1395269]

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT,payment_installment_ratio,install_diff_due_paid
7823706,1395269,307424,38.0,21,-359.0,-365.0,21061.800,45000.0,2.136570,6.0
7823707,1395269,307424,37.0,21,-359.0,-365.0,23938.200,45000.0,1.879841,6.0
7830234,1395269,307424,44.0,24,-269.0,-274.0,24577.695,45000.0,1.830928,5.0
7830235,1395269,307424,43.0,24,-269.0,-274.0,20422.305,45000.0,2.203473,5.0
7944751,1395269,307424,51.0,28,-149.0,-169.0,14123.790,45000.0,3.186114,20.0
...,...,...,...,...,...,...,...,...,...,...
11390329,1395269,307424,42.0,23,-299.0,-305.0,23311.980,45000.0,1.930338,6.0
11390330,1395269,307424,34.0,19,-419.0,-428.0,19123.020,45000.0,2.353185,9.0
11390331,1395269,307424,33.0,19,-419.0,-428.0,25876.980,45000.0,1.738997,9.0
11399046,1395269,307424,9.0,6,-809.0,-813.0,33883.200,45000.0,1.328092,4.0


- it looks like the installments were consistently being overpaid. I assume that if the client overpays, the loan recalculates and so the version number keeps changing

### check final engineered_install

In [133]:
engineered_install

,SK_ID_PREV,SK_ID_CURR,installment_avg_payment_ratio,install_avg_diff_due_paid
0,1000001,158271,1.000000,16.000000
1,1000002,101962,1.000000,19.750000
2,1000003,252457,1.000000,15.333333
3,1000004,260094,1.000000,26.714286
4,1000005,176456,0.909091,8.454545
...,...,...,...,...
997747,2843495,260963,1.000000,3.857143
997748,2843496,425374,0.941176,4.000000
997749,2843497,451578,1.000000,2.900000
997750,2843498,393881,1.000000,12.666667


In [134]:
# groupby SK_ID_PREV, get the mean

engineered_install = engineered_install.groupby('SK_ID_CURR').mean()
engineered_install

,SK_ID_PREV,installment_avg_payment_ratio,install_avg_diff_due_paid
SK_ID_CURR,,,
100001,1.610838e+06,1.000000,5.916667
100002,1.038818e+06,1.000000,20.421053
100003,2.281150e+06,1.000000,7.448413
100004,1.564014e+06,1.000000,7.666667
100005,2.495675e+06,1.000000,23.555556
...,...,...,...
456251,2.248017e+06,1.000000,36.285714
456252,1.503599e+06,1.000000,2.833333
456253,1.794583e+06,0.933333,11.977778


In [135]:
# drop SK_ID_PREV

engineered_install.drop(columns='SK_ID_PREV', inplace=True)

,installment_avg_payment_ratio,install_avg_diff_due_paid
SK_ID_CURR,,
100001,1.000000,5.916667
100002,1.000000,20.421053
100003,1.000000,7.448413
100004,1.000000,7.666667
100005,1.000000,23.555556
...,...,...
456251,1.000000,36.285714
456252,1.000000,2.833333
456253,0.933333,11.977778


In [136]:
# reset index

engineered_install = engineered_install.reset_index()
engineered_install

,SK_ID_CURR,installment_avg_payment_ratio,install_avg_diff_due_paid
0,100001,1.000000,5.916667
1,100002,1.000000,20.421053
2,100003,1.000000,7.448413
3,100004,1.000000,7.666667
4,100005,1.000000,23.555556
...,...,...,...
339582,456251,1.000000,36.285714
339583,456252,1.000000,2.833333
339584,456253,0.933333,11.977778
339585,456254,1.000000,18.611111


In [137]:
# save to csv

# engineered_install.to_csv('../datas/engineered_installments.csv', index=False)